In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import os
os.environ["KERAS_BACKEND"] = "tensorflow" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/math-qsa-dataset/train.csv
/kaggle/input/math-qsa-dataset/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-prize/AIMO Prize - Note on Language and Notation.pdf
/kaggle/input/ai-mathematical-olympiad-prize/train.csv
/kaggle/input/ai-mathematical-olympiad-prize/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/aimo/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/ai-mathematical-olympiad-prize/aimo/__init__.py
/kaggle/input/keras-lib-dataset/keras-3.3.3-py3-none-any.whl
/kaggle/input/keras-lib-dataset/keras_nlp-0.12.1-py3-none-any.whl
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/config.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/assets/tokenizer/vocabulary.spm


In [12]:
!pip install -q /kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl --no-deps
!pip install pylatexenc

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl'



In [13]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/ai-mathematical-olympiad-prize"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train
keras.utils.set_random_seed(CFG.seed)

In [14]:
df1 = pd.read_csv("/kaggle/input/math-qsa-dataset/train.csv")
df2 = pd.read_csv("/kaggle/input/math-qsa-dataset/test.csv")
df = pd.concat([df1, df2], axis=0)

In [15]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)

# Preprocessing Pipeline

In [16]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


text = """
    draw((0,2.5)--(0,7.5)--(4,10)--(8,7.5)--(8,2.5)--(4,0)--cycle,linewidth(1));
    draw((8,2.5)--(11.5,-1)--(9,-5)--(5,-4.5)--(4,0),linewidth(1));
    dot((0,2.5)); dot((0,7.5)); dot ((4,10)); dot((8,7.5)); dot((8,2.5)); dot((4,0));
    label("I",(0,2.5),W); label("H",(0,7.5),W); label("G",(4,10),N);
    label("F",(8,7.5),E); label("E",(8,2.5),NW); label("A",(4,0),SW);
    dot((11.5,-1)); dot((9,-5)); dot((5,-4.5));
    label("D",(11.5,-1),E); label("C",(9,-5),SE); label("B",(5,-4.5),SW);
    [/asy]"""

# Remove the [asy] tags
text = re.sub(r'\[asy\]', '', text)
text = re.sub(r'\[\/asy\]', '', text)

# Define a function to convert coordinates to a readable format
def convert_draw_command(draw_command):
    pattern = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--cycle.*?\);')
    match = pattern.match(draw_command)
    if match:
        coords = match.groups()
        return f"A regular pentagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
    pattern = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\),.*?\);')
    match = pattern.match(draw_command)
    if match:
        coords = match.groups()
        return f"A regular hexagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
    return ""

# Function to convert dot and label commands to readable format
def convert_dot_label_commands(text):
    pattern = re.compile(r'dot\(\((.*?)\)\);')
    text = pattern.sub(r'A point at \1.', text)
    pattern = re.compile(r'label\("(.*?)",\((.*?)\),.*?\);')
    text = pattern.sub(r'The point \1 is at coordinates \2.', text)
    return text


commands = text.split('\n')


readable_text = []
for command in commands:
    if 'draw' in command:
        readable_text.append(convert_draw_command(command))
    else:
        readable_text.append(convert_dot_label_commands(command))


readable_text = ' '.join(readable_text)


sentences = sent_tokenize(readable_text)

# Remove stop words and tokenize the remaining words
stop_words = set(stopwords.words('english'))
filtered_sentences = []

for sentence in sentences:
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words]
    filtered_sentences.append(' '.join(filtered_sentence))


filtered_text = ' '.join(filtered_sentences)

print("Filtered Text: ", filtered_text)


Filtered Text:  point 0,2.5 . point 0,7.5. dot ( ( 4,10 ) ) ; point 8,7.5 . point 8,2.5 . point 4,0 . point coordinates 0,2.5 . point H coordinates 0,7.5 . point G coordinates 4,10 . point F coordinates 8,7.5 . point E coordinates 8,2.5 . point coordinates 4,0 . point 11.5 , -1 . point 9 , -5 . point 5 , -4.5 . point coordinates 11.5 , -1 . point C coordinates 9 , -5 . point B coordinates 5 , -4.5 .


In [17]:
from pylatexenc.latex2text import LatexNodes2Text

def latexconvertor(problems):
    v = []
    for i in range(len(problems)):
        latex = problems[i]
        text = LatexNodes2Text().latex_to_text(latex)
        v.append(text)
    return v


data = latexconvertor(df["problem"])
        

In [18]:
import nltk

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize




stop_words = set(stopwords.words('english'))


word_tokens = word_tokenize(data[4])


filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words]


filtered_text = ' '.join(filtered_sentence)


print("Original Text: ", data[4])
print("Filtered Text: ", filtered_text)


Original Text:  Regular pentagon ABCDE and regular hexagon AEFGHI are drawn on opposite sides of line segment AE such that they are coplanar. What is the degree measure of exterior angle DEF? [asy]
draw((0,2.5)–(0,7.5)–(4,10)–(8,7.5)–(8,2.5)–(4,0)–cycle,linewidth(1));
draw((8,2.5)–(11.5,-1)–(9,-5)–(5,-4.5)–(4,0),linewidth(1));
dot((0,2.5)); dot((0,7.5)); dot ((4,10)); dot((8,7.5)); dot((8,2.5)); dot((4,0));

label("I",(0,2.5),W); label("H",(0,7.5),W); label("G",(4,10),N);

label("F",(8,7.5),E); label("E",(8,2.5),NW); label("A",(4,0),SW);
dot((11.5,-1)); dot((9,-5)); dot((5,-4.5));
label("D",(11.5,-1),E); label("C",(9,-5),SE); label("B",(5,-4.5),SW);
[/asy]
Filtered Text:  Regular pentagon ABCDE regular hexagon AEFGHI drawn opposite sides line segment AE coplanar . degree measure exterior angle DEF ? [ asy ] draw ( ( 0,2.5 ) – ( 0,7.5 ) – ( 4,10 ) – ( 8,7.5 ) – ( 8,2.5 ) – ( 4,0 ) –cycle , linewidth ( 1 ) ) ; draw ( ( 8,2.5 ) – ( 11.5 , -1 ) – ( 9 , -5 ) – ( 5 , -4.5 ) – ( 4,0 ) , linew

In [ ]:
def 

In [ ]:
data

# Prompt Engineering

In [ ]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [ ]:
df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

In [ ]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
# Take a random sample
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))